In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import gbd_mapping as gbd
from db_queries import get_outputs, get_ids
from get_draws.api import get_draws
from vivarium import Artifact

import bep_summarizer

!whoami
!date

ndbs
Tue Feb  4 15:10:31 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

In [50]:
artifact_dir = '/ihme/costeffectiveness/artifacts/vivarium_gates_bep/'
results_dir = '/share/costeffectiveness/results/vivarium_gates_bep/base_model'

locations_rundates = {
    'Mali': '2020_02_03_21_34_18',
    'Tanzania': '2020_02_03_21_34_44'
}

locations = list(locations_rundates.keys())

### Get GBD round id for 2017

In [5]:
get_ids("gbd_round")

,gbd_round_id,gbd_round
0,1,2010
1,2,2013
2,3,2015
3,4,2016
4,5,2017
5,6,2019
6,7,2020


In [21]:
gbd_2017_id = 5

In [8]:
location_ids = get_ids("location")
location_ids.head()

,location_id,location_name,location_description
0,1,Global,None
1,2,Developing,global
2,3,Developed,global
3,4,"Southeast Asia, East Asia, and Oceania",superregion
4,5,East Asia,region


In [43]:
location_ids.loc[location_ids.location_name.str.contains(r"Mali|Tanzania", regex=True)]

,location_id,location_name,location_description
188,189,United Republic of Tanzania,admin0
210,211,Mali,admin0
361,362,Mali Federation,admin0
6492,27867,Malisa Ward,Imported from PHC Nigeria project DHIS datafil...
7392,28767,Dan Maliki Ward,Imported from PHC Nigeria project DHIS datafil...
16689,42199,Malipo Xian,Short name added per INF-4797
24262,50972,Malinaltepec,admin 2. Added per ticket INF-3633
24560,51270,Malinalco,admin 2. Added per ticket INF-3633
30100,56868,Malibcong,Added INF-5109
30101,56869,Malilipot,Added INF-5109


### Get location ids for Mali and Tanzania

In [20]:
location_ids.query("location_name in ['Mali', 'United Republic of Tanzania']")

,location_id,location_name,location_description
188,189,United Republic of Tanzania,admin0
210,211,Mali,admin0


In [22]:
tanzania_id = 189
mali_id = 211

### Get ids for causes

In [23]:
cause_ids = get_ids('cause')
cause_ids.head()

,cause_id,cause_name,acause,cause_description
0,294,All causes,_all,None
1,295,"Communicable, maternal, neonatal, and nutritio...",_comm,None
2,296,HIV/AIDS and tuberculosis,_hivtb,None
3,297,Tuberculosis,tb,None
4,298,HIV/AIDS,hiv,None


In [30]:
sim_causes = cause_ids.query(
    "cause_name in ['All causes', 'Neonatal disorders', "
    + "'Diarrheal diseases', 'Lower respiratory infections',"
    + "'Meningitis', 'Measles',"
    + "'Protein-energy malnutrition']"
)
sim_causes

,cause_id,cause_name,acause,cause_description
0,294,All causes,_all,None
8,302,Diarrheal diseases,diarrhea,None
28,322,Lower respiratory infections,lri,None
38,332,Meningitis,meningitis,None
47,341,Measles,measles,None
86,380,Neonatal disorders,neonatal,None
93,387,Protein-energy malnutrition,nutrition_pem,None


In [31]:
all_causes = 294
diarrhea = 302
lri = 322
meningitis = 332
measles = 341
neonatal = 380
pem = 387

### Get measure ids and metric ids using R shiny tool

https://shiny.ihme.washington.edu/content/88/

### Measures

In [28]:
deaths = 1
prevalence = 5
incidence = 6
emr = 9 # Excess mortality rate
acmr = 14 # All-cause mortality rate
csmr = 15 # Cause-specific mortality rate
relative_risk = 11
birth_prev = 38


### Metrics

In [29]:
number = 1
percent = 2
rate = 3

### Age group ids and sex ids

In [64]:
early_nn = 2
late_nn = 3
post_nn = 4
one_to_four = 5

male=1
female=2
both=3

## Get GBD death rate from PEM in Mali and Tanzania

In [65]:
pem_deaths = get_outputs('cause', cause_id=pem,
                         measure_id=deaths,
                         metric_id=rate,
                         gbd_round_id=gbd_2017_id,
                         location_id=[tanzania_id,mali_id],
                         age_group_id=[early_nn,late_nn,post_nn,one_to_four],
                         sex_id=[male,female],
                        )
pem_deaths

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,measure_name,metric_name,sex,val,upper,lower
0,2,387,189,1,3,1,2017,nutrition_pem,Early Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Male,0.000000,0.000000,0.000000
1,2,387,189,1,3,2,2017,nutrition_pem,Early Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Female,0.000000,0.000000,0.000000
2,3,387,189,1,3,1,2017,nutrition_pem,Late Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Male,0.000000,0.000000,0.000000
3,3,387,189,1,3,2,2017,nutrition_pem,Late Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Female,0.000000,0.000000,0.000000
4,4,387,189,1,3,1,2017,nutrition_pem,Post Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Male,0.001028,0.001748,0.000530
5,4,387,189,1,3,2,2017,nutrition_pem,Post Neonatal,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Female,0.001188,0.001882,0.000721
6,5,387,189,1,3,1,2017,nutrition_pem,1 to 4,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Male,0.000368,0.000610,0.000193
7,5,387,189,1,3,2,2017,nutrition_pem,1 to 4,Protein-energy malnutrition,False,United Republic of Tanzania,Deaths,Rate,Female,0.000456,0.000705,0.000268
8,2,387,211,1,3,1,2017,nutrition_pem,Early Neonatal,Protein-energy malnutrition,False,Mali,Deaths,Rate,Male,0.000000,0.000000,0.000000
9,2,387,211,1,3,2,2017,nutrition_pem,Early Neonatal,Protein-energy malnutrition,False,Mali,Deaths,Rate,Female,0.000000,0.000000,0.000000


## Get all-cause mortality rate

In [66]:
all_cause_deaths = get_outputs('cause', cause_id=all_causes,
                         measure_id=deaths,
                         metric_id=rate,
                         gbd_round_id=gbd_2017_id,
                         location_id=[tanzania_id,mali_id],
                         age_group_id=[early_nn,late_nn,post_nn,one_to_four],
                         sex_id=[male,female],
                        )
all_cause_deaths

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,measure_name,metric_name,sex,val,upper,lower
0,2,294,189,1,3,1,2017,_all,Early Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Male,1.199046,1.585562,0.914621
1,2,294,189,1,3,2,2017,_all,Early Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Female,0.822769,1.034624,0.659351
2,3,294,189,1,3,1,2017,_all,Late Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Male,0.081020,0.106901,0.061894
3,3,294,189,1,3,2,2017,_all,Late Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Female,0.071673,0.090191,0.057467
4,4,294,189,1,3,1,2017,_all,Post Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Male,0.021405,0.028113,0.016398
5,4,294,189,1,3,2,2017,_all,Post Neonatal,All causes,False,United Republic of Tanzania,Deaths,Rate,Female,0.020863,0.026227,0.016770
6,5,294,189,1,3,1,2017,_all,1 to 4,All causes,False,United Republic of Tanzania,Deaths,Rate,Male,0.004757,0.006187,0.003702
7,5,294,189,1,3,2,2017,_all,1 to 4,All causes,False,United Republic of Tanzania,Deaths,Rate,Female,0.004783,0.005939,0.003900
8,2,294,211,1,3,1,2017,_all,Early Neonatal,All causes,False,Mali,Deaths,Rate,Male,1.970545,2.502055,1.564036
9,2,294,211,1,3,2,2017,_all,Early Neonatal,All causes,False,Mali,Deaths,Rate,Female,1.380080,1.672784,1.148452


## See what's in the BEP data artifacts

In [60]:
art_mali = Artifact(artifact_dir + 'mali.hdf', filter_terms=['year_start == 2017', 'age_start < 5'])
print(art_mali)

Artifact containing the following keys:
metadata
	keyspace
	locations
population
	structure
	age_bins
	demographic_dimensions
	theoretical_minimum_risk_life_expectancy
cause
	all_causes
		cause_specific_mortality_rate
	diarrheal_diseases
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	measles
		prevalence
		incidence_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	lower_respiratory_infections
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	meningitis
		prevalence
		incidence_rate
		remission_rate
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	protein_energy_malnutrition
		cause_specific_mortality_rate
		excess_mortality_rate
		disability_weight
		restrictions
	neonatal_disorders
		cause_specific_mortality_rate
		pr

In [62]:
art_mali.load('metadata.locations')

['Mali']

## Check all-cause mortality rate and CSMR for PEM

In [73]:
mali_acmr_draws = art_mali.load('cause.all_causes.cause_specific_mortality_rate')
mali_acmr_draws

draw_0    draw_1  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.772252  1.193572   
                0.019178  0.076712 2017       2018      0.160409  0.106730   
                0.076712  1.000000 2017       2018      0.042451  0.028858   
                1.000000  5.000000 2017       2018      0.017982  0.012360   
         Male   0.000000  0.019178 2017       2018      2.687322  1.642161   
                0.019178  0.076712 2017       2018      0.186199  0.113246   
                0.076712  1.000000 2017       2018      0.050801  0.031332   
                1.000000  5.000000 2017       2018      0.020316  0.012711   

                                                          draw_2    draw_3  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.212082  1.400893   
                0.019178  0.076712 2017       2018      0.108966  0.123290   
                0.076712  1.000000 2017       2018      0.029171  0.033966   
                1.000000  5.000000 2017       2018      0.012404  0.014483   
         Male   0.000000  0.019178 2017       2018      1.672946  2.010108   
                0.019178  0.076712 2017       2018      0.115623  0.137546   
                0.076712  1.000000 2017       2018      0.031766  0.038454   
                1.000000  5.000000 2017       2018      0.012745  0.015619   

                                                          draw_4    draw_5  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.438280  1.478125   
                0.019178  0.076712 2017       2018      0.128783  0.132854   
                0.076712  1.000000 2017       2018      0.034737  0.035614   
                1.000000  5.000000 2017       2018      0.014890  0.015152   
         Male   0.000000  0.019178 2017       2018      2.073504  2.144736   
                0.019178  0.076712 2017       2018      0.143070  0.148220   
                0.076712  1.000000 2017       2018      0.039532  0.040784   
                1.000000  5.000000 2017       2018      0.016069  0.016407   

                                                          draw_6    draw_7  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.563417  1.329597   
                0.019178  0.076712 2017       2018      0.141098  0.120334   
                0.076712  1.000000 2017       2018      0.037567  0.032090   
                1.000000  5.000000 2017       2018      0.015913  0.013838   
         Male   0.000000  0.019178 2017       2018      2.299563  1.877020   
                0.019178  0.076712 2017       2018      0.159172  0.130098   
                0.076712  1.000000 2017       2018      0.043604  0.035752   
                1.000000  5.000000 2017       2018      0.017425  0.014541   

                                                          draw_8    draw_9  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.585070  1.351465   
                0.019178  0.076712 2017       2018      0.144648  0.121635   
                0.076712  1.000000 2017       2018      0.037924  0.032429   
                1.000000  5.000000 2017       2018      0.016039  0.013759   
         Male   0.000000  0.019178 2017       2018      2.337105  1.916905   
                0.019178  0.076712 2017       2018      0.162397  0.132531   
                0.076712  1.000000 2017       2018      0.044111  0.036289   
                1.000000  5.000000 2017       2018      0.017532  0.014540   

                                                        ...  draw_990  \
location sex    age_start age_end  year_start year_end  ...             
Mali   

In [74]:
mali_acmr = mali_acmr_draws.agg(['mean', 'min', 'max'], axis=1)
mali_acmr.index = mali_acmr_draws.index
mali_acmr

mean       min  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      1.380080  1.021181   
                0.019178  0.076712 2017       2018      0.124201  0.090030   
                0.076712  1.000000 2017       2018      0.033241  0.024630   
                1.000000  5.000000 2017       2018      0.014177  0.010589   
         Male   0.000000  0.019178 2017       2018      1.970545  1.350925   
                0.019178  0.076712 2017       2018      0.136225  0.093263   
                0.076712  1.000000 2017       2018      0.037454  0.025713   
                1.000000  5.000000 2017       2018      0.015094  0.010444   

                                                             max  
location sex    age_start age_end  year_start year_end            
Mali     Female 0.000000  0.019178 2017       2018      1.822414  
                0.019178  0.076712 2017       2018      0.165568  
                0.076712  1.000000 2017       2018      0.043738  
                1.000000  5.000000 2017       2018      0.018477  
         Male   0.000000  0.019178 2017       2018      2.781183  
                0.019178  0.076712 2017       2018      0.192949  
                0.076712  1.000000 2017       2018      0.052730  
                1.000000  5.000000 2017       2018      0.021153

In [68]:
art_mali.load('cause.protein_energy_malnutrition.cause_specific_mortality_rate').agg(['mean', 'min', 'max'], axis=1)

mean       min       max
Mali Female 0.000000 0.019178 2017 2018  0.000000  0.000000  0.000000
            0.019178 0.076712 2017 2018  0.000000  0.000000  0.000000
            0.076712 1.000000 2017 2018  0.001743  0.000469  0.005327
            1.000000 5.000000 2017 2018  0.001486  0.000406  0.003476
     Male   0.000000 0.019178 2017 2018  0.000000  0.000000  0.000000
            0.019178 0.076712 2017 2018  0.000000  0.000000  0.000000
            0.076712 1.000000 2017 2018  0.002230  0.000594  0.005401
            1.000000 5.000000 2017 2018  0.001547  0.000463  0.003626

### Try to merge with data from GBD to compare

In [78]:
mali_acmr.reset_index().drop(columns='location')

,sex,age_start,age_end,year_start,year_end,mean,min,max
0,Female,0.000000,0.019178,2017,2018,1.380080,1.021181,1.822414
1,Female,0.019178,0.076712,2017,2018,0.124201,0.090030,0.165568
2,Female,0.076712,1.000000,2017,2018,0.033241,0.024630,0.043738
3,Female,1.000000,5.000000,2017,2018,0.014177,0.010589,0.018477
4,Male,0.000000,0.019178,2017,2018,1.970545,1.350925,2.781183
5,Male,0.019178,0.076712,2017,2018,0.136225,0.093263,0.192949
6,Male,0.076712,1.000000,2017,2018,0.037454,0.025713,0.052730
7,Male,1.000000,5.000000,2017,2018,0.015094,0.010444,0.021153


In [80]:
all_cause_deaths.loc[all_cause_deaths['location_id'] == mali_id,
                     ['location_name', 'cause_name', 'measure_name',
                      'metric_name', 'sex', 'val', 'upper', 'lower']]#.merge(
#mali_acmr.reset_index().drop(columns='location'), on='sex') #ugh, also need to join on age groups...

,location_name,cause_name,measure_name,metric_name,sex,val,upper,lower
8,Mali,All causes,Deaths,Rate,Male,1.970545,2.502055,1.564036
9,Mali,All causes,Deaths,Rate,Female,1.380080,1.672784,1.148452
10,Mali,All causes,Deaths,Rate,Male,0.136225,0.172637,0.108083
11,Mali,All causes,Deaths,Rate,Female,0.124201,0.150363,0.103231
12,Mali,All causes,Deaths,Rate,Male,0.037454,0.047243,0.029791
13,Mali,All causes,Deaths,Rate,Female,0.033241,0.040048,0.027729
14,Mali,All causes,Deaths,Rate,Male,0.015094,0.018912,0.012082
15,Mali,All causes,Deaths,Rate,Female,0.014177,0.017007,0.011861
